# Cuckoo Filter 

### Probabilistic Data Structures
Probabilistic data structures are an important class of data structures that are useful in a variety of applications, particularly in streaming applications and cases that require a  significant amount of space. In general, data structures help organize data so that queries about the  data stored in them can be answered efficiently. Often, these data structures provide exact answers to queries posed about data stored in them. In contrast, probabilistic data structures do not provide exact answers to queries about data stored in them; they typically provide probabilistic estimates to these queries about constituent data. 

### Illustrative Example
To illustrate this point, we will look at the popular use case of testing to see whether an item in a
stream of data has been encountered before; membership testing. In the standard case where the data to 
be analyzed is small enough to fit into memory, each data encountered in the stream can be inserted into
a set data structure. Once we hit an item we would like to test, we can check to see if it is in the set. 
This approach works when the data stream is small enough to fit 
into available memory. However, when we have more data than can fit into memory, it might be cumbersome 
or impossible to use a set for the membership query. Probabilistic data structures are designed to take 
advantage of such situations. Often in computer science, there is a trade-off between time-efficiency 
and the amount of space used. Probabilistic data structures trade off space for exact answers to queries 
about data stored in them. 

### Bloom filter
For the membership query described above, the bloom filter is an example of a probabilistic data 
structure that can provide answers to this query without storing all of the data in a stream. The bloom 
filter leverages hash functions to encode items compactly in an array. Encoded values derived from 
hashing an item to be inserted into a bloom filter can then be used as indices of an array mapping to such data. These 
corresponding indices are then set to indicate that the item has been inserted. Now to check if an item 
has been seen before, the item is hashed and mapped to its expected indices. The obtained indices are 
then checked to see if the values have been set. If all the values have set, then one can say the item 
might have been seen before. However, if any of the values are not set, one can say definitely, that the 
item in question has not been seen before. Hence, membership testing using bloom filters can lead to 
false positives, however, it eliminates false negatives. 

Depending on the capacity (size), number of hash functions, and other specifications for the bloom filter, the false positive rate of a bloom filter can actually be bounded. Traditional bloom filters do not support deletions, since hashing is typically lossy and irreversible; hence deletions or specific manipulations require the entire filter to be rebuilt. However, there have been several modifications proposed that modify traditional bloom filters to allow for deletions, unions, and other set operations. One such example is the counting bloom filter, which extends the bloom filter to allow for deletions. At Fast Forward Labs, bloom filters and a variety of other probabilistic data structures were explored in detail in the report, "Probabilistic Methods for Real-time Streams."

### Overview of cuckoo filter
In this blog post, we focus on a more recent development: the 
cuckoo filter. The cuckoo filter was introduced in a 2014 paper 
as a possible replacement for the bloom filter. It is designed to target applications that store a high number of items with a low false positive rate. Further, the standard cuckoo filter supports dynamic additions and deletions. As noted above, bloom filters 
need to be extended in order to support deletions. An added 
advantage of the bloom filter is that optimized implementations 
use less space than bloom filters while still maintaining low 
positive rates.



The cuckoo filter is backed by a cuckoo hash table and only 
stores fingerprints of items inserted. The fingerprint of an item 
is a reduced bit string derived from the hash of an item to be 
inserted in the cuckoo filter. A cuckoo hash table "consists of 
an array of buckets" where an item to be inserted is mapped to 
two possible buckets based on two hash functions. Each bucket in 
the cuckoo hash table can be configured to store a variable 
number of fingerprints depending on the space and false positive 
considerations desired. Typically, a cuckoo filter is identified 
by the its fingerprint size and bucket size. For example, a (2,4)
cuckoo filter stores 2 bit length fingerprints and each bucket in
the cuckoo hash table can store up to 4 fingerprints. To 
initialize a cuckoo filter, one specifies the capacity of 
desired, the fingerprint size, and the size of the bucket in 
each array.

As noted in the 2014 paper (cite), the cuckoo filter supports 
the standard insert, lookup, and delete operations. Of all these 
operations, the insert operation is most involved. A pseudo code 
of the insert operation is shown below:

{pseudocode for inserting into the cuckoo table}

As indicated in the algorithm above, to insert an item into the 
cuckoo filter, one derives two possible indices from the item 
based on hashing the item and its fingerprint. On obtaining these 
indices, one then inserts the item fingerprint into one of the 
above buckets. As the cuckoo hash table begins to fill up, one 
can encounter a situation where the two possible indices where an 
item can be inserted has been filled, in this case, inserted 
items can then be swapped into other buckets to free up space for 
a new item. As the cuckoo hash table fills up to capacity, more 
items would need to be swapped. To prevent infinite swapping when 
the cuckoo table fills up, one can limit the swapping to be performed.
This number is typically between 400 and 500. 

Lookup and deletions from a cuckoo filter are relatively 
straightforward. To check if an item is in the filter, the item 
is hashed to obtain its fingerprint, and 2 candidate indices. 
Consequently, each index location is check to see if the 
fingerprint of the item is present in the bucket. Deletion 
follows a similar process, but the item's fingerprint is removed 
from the bucket if it is present. 

## Bench Marking against counting bloom filter

To explore the properties of the cuckoo filter, we implemented a 
version of the filter in python. We compare the behavior of this 
filter to a counting bloom filter implementation in python as 
well. While benchmarking is often with fraught with 
intricacies, we present the comparison here to demonstrate the 
capabilities of the cuckoo filter relative to the counting bloom 
filter. For more rigorous benchmarking take a look at the 
referenced paper. 

### Graph of insetion time

### Graph of sizes for different inserts

### false positive for different sizes, extreme case when it is almost full as well. 

### total size benchmark graph

As shown in the different graphs above, the cuckoo filter has several advantages compared to standard bloom filters.
Despite these advantages cuckoo filters are particularly suited to applications where a low false positive rate is desired as well as smaller spacer requirements compared to the bloom filter. 